# Activity: Build a random forest model

## **Introduction**


As you're learning, random forests are popular statistical learning algorithms. Some of their primary benefits include reducing variance, bias, and the chance of overfitting.

This activity is a continuation of the project you began modeling with decision trees for an airline. Here, you will train, tune, and evaluate a random forest model using data from spreadsheet of survey responses from 129,880 customers. It includes data points such as class, flight distance, and inflight entertainment. Your random forest model will be used to predict whether a customer will be satisfied with their flight experience.

**Note:** Because this lab uses a real dataset, this notebook first requires exploratory data analysis, data cleaning, and other manipulations to prepare it for modeling.

## **Step 1: Imports** 


Import relevant Python libraries and modules, including `numpy` and `pandas`libraries for data processing; the `pickle` package to save the model; and the `sklearn` library, containing:
- The module `ensemble`, which has the function `RandomForestClassifier`
- The module `model_selection`, which has the functions `train_test_split`, `PredefinedSplit`, and `GridSearchCV` 
- The module `metrics`, which has the functions `f1_score`, `precision_score`, `recall_score`, and `accuracy_score`


In [1]:
# Import `numpy`, `pandas`, `pickle`, and `sklearn`.
# Import the relevant functions from `sklearn.ensemble`, `sklearn.model_selection`, and `sklearn.metrics`.

### YOUR CODE HERE ###
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,PredefinedSplit,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score,confusion_matrix,ConfusionMatrixDisplay



As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

In [2]:
# RUN THIS CELL TO IMPORT YOUR DATA. 

### YOUR CODE HERE ###

air_data = pd.read_csv("Invistico_Airline.csv")

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `read_csv()` function from the `pandas` library can be helpful here.
 
</details>

Now, you're ready to begin cleaning your data. 

## **Step 2: Data cleaning** 

To get a sense of the data, display the first 10 rows.

In [3]:
# Display first 10 rows.

### YOUR CODE HERE ###
air_data.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `head()` function from the `pandas` library can be helpful here.
 
</details>

Now, display the variable names and their data types. 

In [4]:
# Display variable names and types.

### YOUR CODE HERE ###

air_data.dtypes

satisfaction                          object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
dtype: obj

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

DataFrames have an attribute that outputs variable names and data types in one result.
 
</details>

**Question:** What do you observe about the differences in data types among the variables included in the data?

[Write your response here. Double-click (or enter) to edit.]

Next, to understand the size of the dataset, identify the number of rows and the number of columns.

In [5]:
# Identify the number of rows and the number of columns.

### YOUR CODE HERE ###
air_data.shape

(129880, 22)


<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

There is a method in the `pandas` library that outputs the number of rows and the number of columns in one result.

</details>

Now, check for missing values in the rows of the data. Start with .isna() to get Booleans indicating whether each value in the data is missing. Then, use .any(axis=1) to get Booleans indicating whether there are any missing values along the columns in each row. Finally, use .sum() to get the number of rows that contain missing values.

In [6]:
# Get Booleans to find missing values in data.
# Get Booleans to find missing values along columns.
# Get the number of rows that contain missing values.

### YOUR CODE HERE ###

r_i=[]
for i in air_data.columns:
    print(air_data[i].isna())
    r_i.append(air_data[air_data[i].isna()==True].index.tolist())

r_i=r_i[-1]
print(r_i)
print(len(r_i))


0         False
1         False
2         False
3         False
4         False
          ...  
129875    False
129876    False
129877    False
129878    False
129879    False
Name: satisfaction, Length: 129880, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
129875    False
129876    False
129877    False
129878    False
129879    False
Name: Customer Type, Length: 129880, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
129875    False
129876    False
129877    False
129878    False
129879    False
Name: Age, Length: 129880, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
129875    False
129876    False
129877    False
129878    False
129879    False
Name: Type of Travel, Length: 129880, dtype: bool
0         False
1         False
2         False
3         False
4         False
          ...  
129875    Fals

**Question:** How many rows of data are missing values?**

393 rows of data are missing values

Drop the rows with missing values. This is an important step in data cleaning, as it makes the data more useful for analysis and regression. Then, save the resulting pandas DataFrame in a variable named `air_data_subset`.

In [7]:
# Drop missing values.
# Save the DataFrame in variable `air_data_subset`.

### YOUR CODE HERE ###
air_data_subset=air_data.drop(index=r_i)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

The `dropna()` function is helpful here.
</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The axis parameter passed in to this function should be set to 0 (if you want to drop rows containing missing values) or 1 (if you want to drop columns containing missing values).
</details>

Next, display the first 10 rows to examine the data subset.

In [8]:
# Display the first 10 rows.

### YOUR CODE HERE ###

air_data_subset.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


Confirm that it does not contain any missing values.

In [9]:
# Count of missing values.

### YOUR CODE HERE ###

air_data_subset.isna().sum().sum()

0

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `.isna().sum()` to get the number of missing values for each variable.

</details>

Next, convert the categorical features to indicator (one-hot encoded) features. 

**Note:** The `drop_first` argument can be kept as default (`False`) during one-hot encoding for random forest models, so it does not need to be specified. Also, the target variable, `satisfaction`, does not need to be encoded and will be extracted in a later step.

In [10]:
# Convert categorical features to one-hot encoded features.

### YOUR CODE HERE ###
import re

cat_fe=['Customer Type','Type of Travel','Class']

air_data_subset_dummies=pd.get_dummies(air_data_subset)




<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `pd.get_dummies()` function to convert categorical variables to one-hot encoded variables.
</details>

**Question:** Why is it necessary to convert categorical data into dummy variables?**

It is neceassy to convert catagorical data into dummy vairalbes to find out the correlations in our dataset, for machine learning models to perform mathematical operations 

Next, display the first 10 rows to review the `air_data_subset_dummies`. 

In [11]:
# Display the first 10 rows.

### YOUR CODE HERE ###
air_data_subset_dummies.head(10)


,Age,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,...,Arrival Delay in Minutes,satisfaction_dissatisfied,satisfaction_satisfied,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,65,265,0,0,0,2,2,4,2,3,...,0.0,0,1,1,0,0,1,0,1,0
1,47,2464,0,0,0,3,0,2,2,3,...,305.0,0,1,1,0,0,1,1,0,0
2,15,2138,0,0,0,3,2,0,2,2,...,0.0,0,1,1,0,0,1,0,1,0
3,60,623,0,0,0,3,3,4,3,1,...,0.0,0,1,1,0,0,1,0,1,0
4,70,354,0,0,0,3,4,3,4,2,...,0.0,0,1,1,0,0,1,0,1,0
5,30,1894,0,0,0,3,2,0,2,2,...,0.0,0,1,1,0,0,1,0,1,0
6,66,227,0,0,0,3,2,5,5,5,...,15.0,0,1,1,0,0,1,0,1,0
7,10,1812,0,0,0,3,2,0,2,2,...,0.0,0,1,1,0,0,1,0,1,0
8,56,73,0,0,0,3,5,3,5,4,...,0.0,0,1,1,0,0,1,1,0,0
9,22,1556,0,0,0,3,2,0,2,2,...,26.0,0,1,1,0,0,1,0,1,0


Then, check the variables of air_data_subset_dummies.

In [12]:
# Display variables.

### YOUR CODE HERE ###

air_data_subset_dummies[['satisfaction_dissatisfied','satisfaction_satisfied','Customer Type_Loyal Customer','Customer Type_disloyal Customer','Type of Travel_Business travel','Type of Travel_Personal Travel','Class_Business','Class_Eco','Class_Eco Plus']]

,satisfaction_dissatisfied,satisfaction_satisfied,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,0,1,1,0,0,1,0,1,0
1,0,1,1,0,0,1,1,0,0
2,0,1,1,0,0,1,0,1,0
3,0,1,1,0,0,1,0,1,0
4,0,1,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
129875,0,1,0,1,0,1,0,1,0
129876,1,0,0,1,0,1,1,0,0
129877,1,0,0,1,0,1,0,1,0
129878,1,0,0,1,0,1,0,1,0


**Question:** What changes do you observe after converting the string data to dummy variables?**

[Write your response here. Double-click (or enter) to edit.]

## **Step 3: Model building** 

The first step to building your model is separating the labels (y) from the features (X).

In [13]:
# Separate the dataset into labels (y) and features (X).

### YOUR CODE HERE ###
from sklearn.preprocessing import LabelEncoder

model_l=LabelEncoder()
X=air_data_subset_dummies.drop(['satisfaction_dissatisfied','satisfaction_satisfied'],axis=1)
air_data_subset['satisfaction']=model_l.fit_transform(air_data_subset['satisfaction'])
y=air_data_subset['satisfaction']


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Save the labels (the values in the `satisfaction` column) as `y`.

Save the features as `X`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

To obtain the features, drop the `satisfaction` column from the DataFrame.

</details>

Once separated, split the data into train, validate, and test sets. 

In [14]:
# Separate into train, validate, test sets.

### YOUR CODE HERE ###

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `train_test_split()` function twice to create train/validate/test sets, passing in `random_state` for reproducible results. 

</details>

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Split `X`, `y` to get `X_train`, `X_test`, `y_train`, `y_test`. Set the `test_size` argument to the proportion of data points you want to select for testing. 

Split `X_train`, `y_train` to get `X_tr`, `X_val`, `y_tr`, `y_val`. Set the `test_size` argument to the proportion of data points you want to select for validation. 

</details>

### Tune the model

Now, fit and tune a random forest model with separate validation set. Begin by determining a set of hyperparameters for tuning the model using GridSearchCV.


In [15]:
# Determine set of hyperparameters.

### YOUR CODE HERE ###

from sklearn.preprocessing import LabelEncoder

rf=RandomForestClassifier(n_estimators=300,max_features=18)

rf.fit(x_train,y_train)
print(rf.score)
dir(rf)

<bound method ClassifierMixin.score of RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=18,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)>


['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_make_estimator',
 '_more_tags',
 '_required_parameters',
 '_set_oob_score',
 '_validate_X_predict',
 '_validate_estimator',
 '_validate_y_class_weight',
 'apply',
 'base_estimator',
 'base_estimator_',
 'bootstrap',
 'ccp_alpha',
 'class_weight',
 'classes_',
 'criterion',
 'decision_path',
 'estimator_params',
 'estimators_',
 'feature_importances_',
 'fit',
 'get_params',
 'max_depth',
 'max_features',
 'max_leaf_nodes',
 'max_samples',
 'min_impurity_d

In [16]:
y_pred=rf.predict(x_test)



In [17]:
print('Accuracy Score:','%.3f' % accuracy_score(y_test,y_pred), '\n'
      'Precision Score:','%.3f'% precision_score(y_test,y_pred),'\n'
      'recall Score:','%.3f' % recall_score(y_test,y_pred),'\n'
      'f1 Score:','%.3f' % f1_score(y_test,y_pred))

Accuracy Score: 0.956 
Precision Score: 0.967 
recall Score: 0.951 
f1 Score: 0.959


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Create a dictionary `cv_params` that maps each hyperparameter name to a list of values. The GridSearch you conduct will set the hyperparameter to each possible value, as specified, and determine which value is optimal.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The main hyperparameters here include `'n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'max_features', and 'max_samples'`. These will be the keys in the dictionary `cv_params`.

</details>

Next, create a list of split indices.

In [19]:
# Create list of split indices.

### YOUR CODE HERE ###

parms={'min_samples_leaf':[12,1,9,7,6],'min_samples_split':[1,67,8],'n_estimators':[21,45],'max_features':[18,5,6,9]}
scoring = {'accuracy', 'precision', 'recall', 'f1'}

model_g=GridSearchCV(rf,parms,cv=5,scoring=scoring,refit='f1')
model_g.fit(x_train,y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=18,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=300, n_jobs=None,
                                              oob_score=False,
                                              random_st

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use list comprehension, iterating over the indices of `X_train`. The list can consists of 0s to indicate data points that should be treated as validation data and -1s to indicate data points that should be treated as training data.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Use `PredfinedSplit()`, passing in `split_index`, saving the output as `custom_split`. This will serve as a custom split that will identify which data points from the train set should be treated as validation data during GridSearch.

</details>

Now, instantiate your model.

In [ ]:
# Instantiate model.

### YOUR CODE HERE ### 


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results. This will help you instantiate a random forest model, `rf`.

</details>

Next, use GridSearchCV to search over the specified parameters.

In [ ]:
# Search over specified parameters.

### YOUR CODE HERE ### 

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `GridSearchCV()`, passing in `rf` and `cv_params` and specifying `cv` as `custom_split`. Additional arguments that you can specify include: `refit='f1', n_jobs = -1, verbose = 1`. 

</details>

Now, fit your model.

In [21]:
# Fit the model.

### YOUR CODE HERE ###

model_g.cv_results_

{'mean_fit_time': array([0.02558503, 0.03205786, 3.63586259, 7.66987834, 3.64697027,
        8.01334262, 0.02576776, 0.03307071, 3.83920579, 8.20889535,
        4.14435883, 8.79972305, 0.02524352, 0.03528752, 3.70701823,
        7.8379488 , 3.782692  , 8.05311203, 0.02498312, 0.03281817,
        3.69666557, 7.85628295, 3.85055938, 8.34801478, 0.02467961,
        0.03341279, 3.75529971, 8.07145047, 3.94602432, 8.59401016,
        0.0280406 , 0.03536868, 1.23942051, 2.60519633, 1.29802003,
        2.80017738, 0.02603245, 0.03396621, 1.29512677, 2.7619925 ,
        1.40560145, 3.03874397, 0.02624955, 0.03324928, 1.25189347,
        2.74699855, 1.36852193, 2.83755407, 0.02537503, 0.03575373,
        1.32080345, 2.88796759, 1.45577807, 3.07997999, 0.02793493,
        0.03856206, 1.34365296, 2.82764907, 1.37852378, 2.9664978 ,
        0.02580962, 0.03284235, 1.46860447, 3.1479825 , 1.58953242,
        3.25413237, 0.02459168, 0.0361258 , 1.53669319, 3.26783724,
        1.60984454, 3.40145316,

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train the GridSearchCV model on `X_train` and `y_train`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Add the magic function `%%time` to keep track of the amount of time it takes to fit the model and display this information once execution has completed. Remember that this code must be the first line in the cell.

</details>

Finally, obtain the optimal parameters.

In [22]:
# Obtain optimal parameters.

### YOUR CODE HERE ###

model_g.best_estimator_


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=45,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
model_g.best_index_

101

In [24]:
model_g.best_params_

{'max_features': 9,
 'min_samples_leaf': 1,
 'min_samples_split': 8,
 'n_estimators': 45}

In [25]:

model_g.best_score_

0.9570211104475419

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `best_params_` attribute to obtain the optimal values for the hyperparameters from the GridSearchCV model.

</details>

## **Step 4: Results and evaluation** 

Use the selected model to predict on your test data. Use the optimal parameters found via GridSearchCV.

In [ ]:
# Use optimal parameters on GridSearchCV.

### YOUR CODE HERE ###


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results and passing in the optimal hyperparameters found in the previous step. To distinguish this from the previous random forest model, consider naming this variable `rf_opt`.

</details>

Once again, fit the optimal model.

In [26]:
# Fit the optimal model.

### YOUR CODE HERE ###

model_g.best_estimator_.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=9,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=45,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train `rf_opt` on `X_train` and `y_train`.

</details>

And predict on the test set using the optimal model.

In [ ]:
# Predict on test set.

### YOUR CODE HERE ###

y_pred=model_g.best_estimator_.predict(y_test)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `predict()` function to make predictions on `X_test` using `rf_opt`. Save the predictions now (for example, as `y_pred`), to use them later for comparing to the true labels. 

</details>

### Obtain performance scores

First, get your precision score.

In [28]:
# Get Accuracy score.

### YOUR CODE HERE ###

print('Accuracy Score:','%.3f' % accuracy_score(y_test,y_pred))

Accuracy Score: 0.956


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `precision_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Then, collect the recall score.

In [29]:
# Get Precision score.

### YOUR CODE HERE ###
print('Precision Score:','%.3f'% precision_score(y_test,y_pred))

Precision Score: 0.967


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `recall_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Next, obtain your accuracy score.

In [30]:
# Get recall score.

### YOUR CODE HERE ###
print('recall Score:','%.3f' % recall_score(y_test,y_pred))

recall Score: 0.951


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `accuracy_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Finally, collect your F1-score.

In [31]:
# Get F1 score.

### YOUR CODE HERE ###
print('f1 Score:','%.3f' % f1_score(y_test,y_pred))

f1 Score: 0.959


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `f1_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

**Question:** How is the F1-score calculated?

Harmonic mean of precision and recall 

**Question:** What are the pros and cons of performing the model selection using test data instead of a separate validation dataset?

[Write your response here. Double-click (or enter) to edit.]



### Evaluate the model

Now that you have results, evaluate the model. 

**Question:** What are the four basic parameters for evaluating the performance of a classification model?

[Write your response here. Double-click (or enter) to edit.]

**Question:**  What do the four scores demonstrate about your model, and how do you calculate them?

[Write your response here. Double-click (or enter) to edit.]

Calculate the scores: precision score, recall score, accuracy score, F1 score.

In [ ]:
# Precision score on test data set.

### YOUR CODE HERE ###


In [ ]:
# Recall score on test data set.

### YOUR CODE HERE ###


In [ ]:
# Accuracy score on test data set.

### YOUR CODE HERE ###


In [ ]:
# F1 score on test data set.

### YOUR CODE HERE ###


**Question:** How does this model perform based on the four scores?

[Write your response here. Double-click (or enter) to edit.]

### Evaluate the model

Finally, create a table of results that you can use to evaluate the performace of your model.

In [40]:
# Create table of results.

### YOUR CODE HERE ###


def make_results(model_object):
    results=model_object.cv_results_
    cols=[i for i in results.keys() if i.startswith('mean')]
    mean_cols=[results[i] for i in cols]
    mean_cols=[list(i) for i in mean_cols]
    res_2=np.array(mean_cols)
    #res_2=res_2.reshape(res_2.shape[1],res_2.shape[0])
    print(res_2)
    results_df=pd.DataFrame(res_2,cols)
    return results_df[results_df['mean_test_f1']==results_df['mean_test_f1'].max()]

In [41]:
make_results(model_g)

[[0.02558503 0.03205786 3.63586259 7.66987834 3.64697027 8.01334262
  0.02576776 0.03307071 3.83920579 8.20889535 4.14435883 8.79972305
  0.02524352 0.03528752 3.70701823 7.8379488  3.782692   8.05311203
  0.02498312 0.03281817 3.69666557 7.85628295 3.85055938 8.34801478
  0.02467961 0.03341279 3.75529971 8.07145047 3.94602432 8.59401016
  0.0280406  0.03536868 1.23942051 2.60519633 1.29802003 2.80017738
  0.02603245 0.03396621 1.29512677 2.7619925  1.40560145 3.03874397
  0.02624955 0.03324928 1.25189347 2.74699855 1.36852193 2.83755407
  0.02537503 0.03575373 1.32080345 2.88796759 1.45577807 3.07997999
  0.02793493 0.03856206 1.34365296 2.82764907 1.37852378 2.9664978
  0.02580962 0.03284235 1.46860447 3.1479825  1.58953242 3.25413237
  0.02459168 0.0361258  1.53669319 3.26783724 1.60984454 3.40145316
  0.02448182 0.03419681 1.43813767 3.0282227  1.51087728 3.21444702
  0.02573237 0.03265605 1.44081764 3.10678959 1.53867164 3.25699749
  0.02449474 0.03246951 1.42361941 3.11509032 1.5

KeyError: 'mean_test_f1'


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Build a table to compare the performance of the models. Create a DataFrame using the `pd.DataFrame()` function.

</details>

**Question:** How does the random forest model compare to the decision tree model you built in the previous lab?

[Write your response here. Double-click (or enter) to edit.]



## **Considerations**


**What are the key takeaways from this lab? Consider important steps when building a model, most effective approaches and tools, and overall results.**

[Write your response here. Double-click (or enter) to edit.]


**What summary would you provide to stakeholders?**

[Write your response here. Double-click (or enter) to edit.]

### References

[What is the Difference Between Test and Validation Datasets?,  Jason Brownlee](https://machinelearningmastery.com/difference-test-validation-datasets/)

[Decision Trees and Random Forests Neil Liberman](https://towardsdatascience.com/decision-trees-and-random-forests-df0c3123f991)

**Congratulations!** You've completed this lab. However, you may not notice a green check mark next to this item on Coursera's platform. Please continue your progress regardless of the check mark. Just click on the "save" icon at the top of this notebook to ensure your work has been logged